# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [2]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [3]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [4]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [5]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [6]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [7]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [10]:
def jacobi(A, b, niter = 50, eps = 1e-5):
    
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    D = np.diag(diag_1d)
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    c = invD @ b
    n = 10
    x0 = np.ones(n)
    x = x0
    for _ in range(niter):
        x = BB @ x + c
    return x

x = jacobi(A,b)
print(x)
A @ x - b

[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]


array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

In [26]:
# Now make the elements of A "smaller"
def jacobi_mod(A, b, niter = 50, eps = 1e-5):
    '''Same inputs, but implementation, but returns different items for comparison
        Compute the Jacobi iteration
        Compute the normal of B
        Compute the actual solution using package solutions and return the difference.
        Returns:
        x: the solved matrix
        normB: the normal of the B matrix
        xx: the solution by numpypackages, used for the difference'''
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    D = np.diag(diag_1d)
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    # added normal computation
    normB = np.linalg.norm(BB)
    c = invD @ b
    n = 10
    x0 = np.ones(n)
    x = x0
    for _ in range(niter):
        x = BB @ x + c
    xx = np.linalg.solve(A, b)
    return x, normB, (x - xx)
for i in range(1,10):
    # make the elements of A smaller with each iteration
    A1 = rndm.uniform(size=(n, n)) + np.diagflat([15/i]*(n))
    x1, normB, x2 = jacobi_mod(A1,b)
    print("x by Jacobi:", x)
    print("Normal of B:", normB)
    print("Difference betweeen computed and actual:", x2)
    

x by Jacobi: [ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
Normal of B: 0.3566914437228362
Difference betweeen computed and actual: [-6.93889390e-18  0.00000000e+00  6.93889390e-18  0.00000000e+00
  0.00000000e+00 -4.33680869e-19 -4.33680869e-19  1.04083409e-17
  0.00000000e+00 -1.38777878e-17]
x by Jacobi: [ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
Normal of B: 0.5932879674772259
Difference betweeen computed and actual: [5.55111512e-17 7.63278329e-17 4.16333634e-17 8.32667268e-17
 5.55111512e-17 6.24500451e-17 4.68375339e-17 7.63278329e-17
 7.63278329e-17 6.93889390e-17]
x by Jacobi: [ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
Normal of B: 0.966109731961311
Difference betweeen computed and actual: [1.26186969e-06 1.88531294e-06 1.50905278e-06 7.15806424

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [31]:
def seidels(A, b, niter = 50, eps = 1e-05):
    x = np.ones(b.shape[0])
    
    for i in range(n_iter):
        for k in range(b.shape[0]):
              x[k] = (b[k]- (A[k][:k] @ x[:k]) - (A[k][k+1:] @ x[k+1:])) / A[k,k]
    return x

x = seidels(A, b)
print(x)
A @ x - b

[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]


array([ 0.00000000e+00,  1.11022302e-16,  1.11022302e-16,  0.00000000e+00,
       -1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        2.77555756e-17,  1.11022302e-16])

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [32]:
def minres(A, b, niter = 50):
    x = np.ones(b.shape[0])
    for i in range(niter):
        r = A @ x - b
        k = (r @ A @ r) / np.linalg.norm(A @ r)**2
        x = x - k * r     
    return x

x = minres(A, b)
x1 = np.linalg.solve(A, b)
print(x)
print(x1)

[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
